In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f, Cylinder
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../functions.jl")

generate_q_init (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [13]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

In [14]:
add_coordinate!(robot, FrameOrigin("rh_ffdistal"); id="rh_ffdistal")
add_coordinate!(robot, FrameOrigin("rh_mfdistal"); id="rh_mfdistal")
add_coordinate!(robot, FrameOrigin("rh_rfdistal"); id="rh_rfdistal")
add_coordinate!(robot, FrameOrigin("rh_lfdistal"); id="rh_lfdistal")
add_coordinate!(robot, FrameOrigin("rh_thdistal"); id="rh_thdistal")

"rh_thdistal"

### Hand Visualization

In [4]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.033, -0.01, 0.38]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.033
 0.0  1.0  0.0  -0.01
 0.0  0.0  1.0   0.38
 0.0  0.0  0.0   1.0

In [5]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "rh_ffmiddle")
display_frame(m, ls, "world")
#display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [47]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), frames(robot))

33-element Vector{String}:
 "world"
 "rh_forearm"
 "rh_wrist"
 "rh_palm"
 "rh_manipulator"
 "rh_imu"
 "rh_ffknuckle"
 "rh_ffproximal"
 "rh_ffmiddle"
 "rh_ffdistal"
 "rh_fftip"
 "rh_mfknuckle"
 "rh_mfproximal"
 "rh_mfmiddle"
 "rh_mfdistal"
 "rh_mftip"
 "rh_rfknuckle"
 "rh_rfproximal"
 "rh_rfmiddle"
 "rh_rfdistal"
 "rh_rftip"
 "rh_lfmetacarpal"
 "rh_lfknuckle"
 "rh_lfproximal"
 "rh_lfmiddle"
 "rh_lfdistal"
 "rh_lftip"
 "rh_thbase"
 "rh_thproximal"
 "rh_thhub"
 "rh_thmiddle"
 "rh_thdistal"
 "rh_thtip"

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [15]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Creation of the ball prismatic joints

In [ ]:
vm = Mechanism{Float64}("VirtualBall")

ball_radius = 0.045
ball_position = SVector(0.0, -0.035, 0.32)

attracted_frames = ("rh_ffdistal_mass_coord", "rh_mfdistal_mass_coord", "rh_rfdistal_mass_coord","rh_lfdistal_mass_coord" , 
                    "rh_thdistal_mass_coord", "rh_ffmiddle_mass_coord","rh_mfmiddle_mass_coord", "rh_rfmiddle_mass_coord",
                    "rh_lfmiddle_mass_coord",  "rh_thmiddle_mass_coord", "rh_ffproximal_mass_coord", "rh_mfproximal_mass_coord",
                    "rh_rfproximal_mass_coord", "rh_lfproximal_mass_coord", "rh_thproximal_mass_coord", "rh_palm_mass_coord")
attracted_frames_names = ("ffdistal", "mfdistal", "rfdistal", "lfdistal", "thdistal", "ffmiddle", "mfmiddle", "rfmiddle", "lfmiddle", "thmiddle", "ffprox", 
                "mfprox", "rfprox", "lfprox", "thprox", "palm")


for i in 1:length(attracted_frames)
    add_frame!(vm; id = "revo_frame_1_$(attracted_frames_names[i])")
    add_joint!(vm, Revolute(SVector(0.0,0.0,1.0),Transform(SVector(ball_position[1], ball_position[2], ball_position[3]))); parent=root_frame(vm), child="revo_frame_1_$(attracted_frames_names[i])", id="revo_joint_1_$(attracted_frames_names[i])")
    add_frame!(vm; id="revo_frame_2_$(attracted_frames_names[i])")
    add_joint!(vm, Revolute(SVector(1.0,0.0,0.0)); parent="revo_frame_1_$(attracted_frames_names[i])", child="revo_frame_2_$(attracted_frames_names[i])", id="revo_joint_2_$(attracted_frames_names[i])")
    add_frame!(vm; id="ee_frame_$(attracted_frames_names[i])")
    add_joint!(vm, Rigid(Transform(SVector(0.0,0.0,ball_radius))); parent ="revo_frame_2_$(attracted_frames_names[i])", child ="ee_frame_$(attracted_frames_names[i])", id = "fixed_joint_$(attracted_frames_names[i])")

    add_coordinate!(vm, FrameOrigin("ee_frame_$(attracted_frames_names[i])"); id="$(attracted_frames_names[i]) ee position")
    add_component!(vm, PointMass(0.01, "$(attracted_frames_names[i]) ee position"); id="$(attracted_frames_names[i]) ee mass")

    add_coordinate!(vm, JointSubspace("revo_joint_1_$(attracted_frames_names[i])"); id="revo_joint_1_$(attracted_frames_names[i])")
    #add_component!(vm, LinearDamper(10.0, "revo_joint_1_$(attracted_frames_names[i])"); id="revo_joint_1_$(attracted_frames_names[i])_damper")
    add_coordinate!(vm, JointSubspace("revo_joint_2_$(attracted_frames_names[i])"); id="revo_joint_2_$(attracted_frames_names[i])")
    #add_component!(vm, LinearDamper(0.5, "revo_joint_2_$(attracted_frames_names[i])"); id="revo_joint_2_$(attracted_frames_names[i])_damper")   
    
    I_mat = @SMatrix [0.1  0.    0.  ;0.    0.1  0.  ;0.    0.    0.1]
    add_inertia!(vm, "ee_frame_$(attracted_frames_names[i])", I_mat; id="$(attracted_frames_names[i]) ee inertia")
end


add_gravity_compensation!(vm, VMRobotControl.DEFAULT_GRAVITY)

### Addition of the multiples springs/dampers

Hand motion

In [17]:
vms = VirtualMechanismSystem("myShadowVMS", robot, vm)
root = root_frame(vms.robot)

D = SMatrix{3, 3}(50., 0., 0., 0., 50.0, 0., 0., 0., 50.)

stiffnesses = [30.0, 30.0, 30.0, 30.0, 30.0, 50.0, 50.0, 50.0, 50.0, 50.0, 100.0, 100.0, 100.0, 100.0, 100.0, 50.0]

# hand converging to the ball 

for i in 1:length(attracted_frames)
    K = SMatrix{3, 3}(stiffnesses[i], 0., 0., 0., stiffnesses[i], 0., 0., 0., stiffnesses[i])
    add_coordinate!(vms, CoordDifference(".virtual_mechanism.$(attracted_frames_names[i]) ee position", ".robot.$(attracted_frames[i])"); id = "ee $(attracted_frames_names[i]) diff")
    add_component!(vms, LinearSpring(K, "ee $(attracted_frames_names[i]) diff"); id = "ee $(attracted_frames_names[i]) spring")
    add_component!(vms, LinearDamper(D, "ee $(attracted_frames_names[i]) diff"); id = "ee $(attracted_frames_names[i]) damper")
end

# fingers spacing : Joint level

add_coordinate!(vms, ConstCoord(0.6); id = "angular spring length")

#ff mf spacing
add_coordinate!(vms, CoordDifference(".robot.rh_MFJ4_coord", ".robot.rh_FFJ4_coord"); id="ff mf j4 angular diff")
add_coordinate!(vms, CoordDifference("ff mf j4 angular diff", "angular spring length") ; id="ff mf j4 angular error")
add_component!(vms, LinearSpring(10.0, "ff mf j4 angular error"); id="ff mf angular spring")
add_component!(vms, LinearDamper(10.0, "ff mf j4 angular error"); id="ff mf angular damper")

#mf rf spacing
add_coordinate!(vms, CoordSum(".robot.rh_RFJ4_coord", ".robot.rh_MFJ4_coord"); id="mf rf j4 angular diff")
add_coordinate!(vms, CoordSum("mf rf j4 angular diff", "angular spring length") ; id="mf rf j4 angular error")
add_component!(vms, LinearSpring(10.0, "mf rf j4 angular error"); id="mf rf angular spring")
add_component!(vms, LinearDamper(10.0, "mf rf j4 angular error"); id="mf rf angular damper")

#rf lf spacing
add_coordinate!(vms, CoordDifference(".robot.rh_RFJ4_coord", ".robot.rh_LFJ4_coord"); id="rf lf j4 angular diff")
add_coordinate!(vms, CoordDifference("rf lf j4 angular diff", "angular spring length") ; id="rf lf j4 angular error")
add_component!(vms, LinearSpring(10.0, "rf lf j4 angular error"); id="rf lf angular spring")
add_component!(vms, LinearDamper(10.0, "rf lf j4 angular error"); id="rf lf angular damper")

#th spacing
add_coordinate!(vms, ConstCoord(1.22); id="th spring length")
add_coordinate!(vms, CoordDifference(".robot.rh_THJ4_coord", "th spring length"); id="th j4 error")
add_component!(vms, LinearSpring(1.0, "th j4 error"); id="th j4 spring")

# finger spacing : finger level 

add_coordinate!(vms, ConstCoord(0.05); id = "finger spring length")

finger_spacing_stiffness = 70.0

add_coordinate!(vms, RotatedCoord(".robot.rh_mfmiddle_mass_coord", ".robot.rh_ffmiddle"); id="ff mf middle diff")
add_coordinate!(vms, CoordSlice("ff mf middle diff", SVector(1)); id="ff mf middle diff x")
add_coordinate!(vms, CoordNorm("ff mf middle diff x"); id="ff mf middle norm")
add_coordinate!(vms, CoordDifference("ff mf middle norm", "finger spring length"); id="ff mf middle error")
add_component!(vms, LinearSpring(finger_spacing_stiffness, "ff mf middle error"); id="ff mf middle spring")
add_component!(vms, LinearDamper(10.0, "ff mf middle error"); id="ff mf middle damper")

add_coordinate!(vms, RotatedCoord(".robot.rh_rfmiddle_mass_coord", ".robot.rh_mfmiddle"); id="mf rf middle diff")
add_coordinate!(vms, CoordSlice("mf rf middle diff", SVector(1)); id="mf rf middle diff x")
add_coordinate!(vms, CoordNorm("mf rf middle diff x"); id="mf rf middle norm")
add_coordinate!(vms, CoordDifference("mf rf middle norm", "finger spring length"); id="mf rf middle error")
add_component!(vms, LinearSpring(finger_spacing_stiffness, "mf rf middle error"); id="mf rf middle spring")
add_component!(vms, LinearDamper(10.0, "mf rf middle error"); id="mf rf middle damper")

add_coordinate!(vms, RotatedCoord(".robot.rh_lfmiddle_mass_coord", ".robot.rh_rfmiddle"); id="rf lf middle diff")
add_coordinate!(vms, CoordSlice("rf lf middle diff", SVector(1)); id="rf lf middle diff x")
add_coordinate!(vms, CoordNorm("rf lf middle diff x"); id="rf lf middle norm")
add_coordinate!(vms, CoordDifference("rf lf middle norm", "finger spring length"); id="rf lf middle error")
add_component!(vms, LinearSpring(finger_spacing_stiffness, "rf lf middle error"); id="rf lf middle spring")
add_component!(vms, LinearDamper(10.0, "rf lf middle error"); id="rf lf middle damper")


"rf lf middle damper"

Ball collision model

In [18]:
add_coordinate!(vms,  ConstCoord(ball_position);  id="ball position")
add_coordinate!(vms, ConstCoord(ball_radius); id="ball radius")

add_coordinate!(vms, FramePoint(".robot.rh_palm", SVector(0. , 0., 0.07)); id="second palm point")

repulsed_frames = (".robot.rh_fftip_mass_coord", ".robot.rh_mftip_mass_coord", ".robot.rh_rftip_mass_coord",".robot.rh_lftip_mass_coord" , 
                    ".robot.rh_thtip_mass_coord", ".robot.rh_ffmiddle_mass_coord",".robot.rh_mfmiddle_mass_coord", ".robot.rh_rfmiddle_mass_coord",
                    ".robot.rh_lfmiddle_mass_coord",  ".robot.rh_thmiddle_mass_coord", ".robot.rh_ffproximal_mass_coord", ".robot.rh_mfproximal_mass_coord",
                    ".robot.rh_rfproximal_mass_coord", ".robot.rh_lfproximal_mass_coord", ".robot.rh_thproximal_mass_coord", ".robot.rh_palm_mass_coord", "second palm point",
                    ".robot.rh_ffdistal", ".robot.rh_mfdistal", ".robot.rh_rfdistal", ".robot.rh_lfdistal", ".robot.rh_thdistal")
repulsed_frames_names = ("fftip", "mftip", "rftip", "lftip", "thtip", "ffmiddle", "mfmiddle", "rfmiddle", "lfmiddle", "thmiddle", "ffprox", 
                "mfprox", "rfprox", "lfprox", "thprox", "palm", "palm2", "ffdistal", "mfdistal", "rfdistal", "lfdistal", "thdistal")

for i in 1:length(repulsed_frames)
    frame = repulsed_frames[i]
    add_coordinate!(vms, CoordDifference(frame, "ball position") ; id = "$(repulsed_frames_names[i]) ball error" )
    add_coordinate!(vms, CoordNorm("$(repulsed_frames_names[i]) ball error") ; id = "$(repulsed_frames_names[i]) ball error norm")
    add_coordinate!(vms, CoordDifference("$(repulsed_frames_names[i]) ball error norm", "ball radius"); id = "shifted $(repulsed_frames_names[i]) ball error" )

    add_component!(vms, ReLUSpring(1000.0, "shifted $(repulsed_frames_names[i]) ball error", true); id="$(repulsed_frames_names[i]) ball repulsive spring")
    add_component!(vms, RectifiedDamper(100.0, "$(repulsed_frames_names[i]) ball error norm", (0.0, 1.1*ball_radius), true, false); id="$(repulsed_frames_names[i]) ball damper")
end

## Simulating the Robot

### Setting Up the Simulation

In [19]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())


tspan = (0., 15.)
vms_compiled = compile(vms)

q_init = zeros(24)
q_init[21] = 1.2
q = (q_init, zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan)
@info "Simulating shadow robot with ball object centric motion"

sol = solve(prob, Rosenbrock23(autodiff=false), progress=true; maxiters=1e6, abstol=1e-3, reltol=1e-3);

[ Info: Simulating shadow robot with ball object centric motion
ODE   0%|█                                              |  ETA: N/A
ODE  79%|██████████████████████████████████████         |  ETA: 0:00:07
ODE 100%|███████████████████████████████████████████████| Time: 0:00:29


### Visualizing the Results

In [20]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)

plotting_vm_kcache = map(plotting_kcache) do k
    VMRobotControl.virtual_mechanism_cache(k)
end
robotsketch!(ls, plotting_vm_kcache; scale = 0.05)

sphere = Sphere(Point3f(ball_position),ball_radius)
mesh!(ls, sphere; color=:magenta, transparency=true)

savepath = joinpath(module_path, "docs/src/assets/shadowhand_object_centric_ball.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath;fps=20,  t=plotting_t);

LoadError: Screen not open!